# Lightweight Fine-Tuning Project

* PEFT technique: LoRA (Low-Rank Adaptation)
    * LoRA is lightweight and memory-efficient — ideal for fine-tuning large models on limited hardware.
    * It freezes most of the model’s parameters and learns small low-rank update matrices, drastically reducing trainable parameters.
    * It preserves the base model’s knowledge while adapting it to your downstream task.  

* Model: "distilbert-base-uncased"
    * A compact, distilled version of BERT — efficient but powerful enough for text classification and other NLP tasks.

* Evaluation approach: Baseline vs. Fine-tuned Performance Comparison

* Fine-tuning dataset: "imdb"

### Step 0: GPU or CPU Support

In [1]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. PyTorch can use the GPU.")
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    print(f"Current GPU name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Training will run on CPU.")

CUDA is available. PyTorch can use the GPU.
Number of GPUs available: 1
Current GPU name: NVIDIA GeForce RTX 5070 Laptop GPU


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Step 1: Load the Foundation Model

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Define label mappings
num_labels = 2
id2label = {0: 'negative', 1: 'positive'}
label2id = {'negative': 0, 'positive': 1}

# Load tokenizer and model
# This is a binary classification
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

# Freeze the base model parameters
for param in model.base_model.parameters():
    param.requires_grad = False

# Print model parameters
total_params = sum(p.numel() for p in model.parameters())
# Print our model parameters
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_params:,} total parameters.")
print(f"{total_trainable_params:,} trainable parameters.")
print(f"{total_trainable_params/total_params:.2%} of parameters are trainable.")

/home/developer/reference/lightweight-fine-tuning-foundation-model.git/master/.micromamba/envs/jupyterlab/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


66,955,010 total parameters.
592,130 trainable parameters.
0.88% of parameters are trainable.


### Step 2: Load the Dataset

In [3]:
from datasets import load_dataset

# Load IMDb dataset
dataset = load_dataset("imdb")

# Check structure
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


### Step 3: Tokenize the Dataset

In [4]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length")

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Verify tokenized dataset
print("Tokenized dataset sample:", tokenized_datasets["train"][0])

Tokenized dataset sample: {'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity

### Step 4: Evalutaion the Foundation Model

In [5]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
import datetime

# Evaluation metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Prepare for training and evaluation
training_args = TrainingArguments(
    output_dir=f"./results/{model_name}/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
    num_train_epochs=5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

In [6]:
trainer.evaluate()

{'eval_loss': 0.6968079805374146,
 'eval_model_preparation_time': 0.0012,
 'eval_accuracy': 0.5,
 'eval_runtime': 329.9441,
 'eval_samples_per_second': 75.77,
 'eval_steps_per_second': 1.185}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

### Step 1: Load Model and Tokenizer

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import numpy as np

# Define label mappings
num_labels = 2
id2label = {0: 'negative', 1: 'positive'}
label2id = {'negative': 0, 'positive': 1}

# Load tokenizer and model
# This is a binary classification
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

/home/developer/reference/lightweight-fine-tuning-foundation-model.git/master/.micromamba/envs/jupyterlab/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Step 2: Apply LoRA PEFT

In [2]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,                 # rank
    lora_alpha=16,       # scaling
    target_modules=["q_lin", "v_lin"],  # attention projection layers
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(base_model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


### Step 3: Load and Tokenize Dataset

In [3]:
# Merge loading and Tokenizing, we've shown how to load and tokenize separately
from datasets import load_dataset

dataset = load_dataset("imdb")

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length")

tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5000))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

### Step 4: Define Evaluation Metrics

In [4]:
import numpy as np

# Evaluation metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

### Step 5: Training Prep and Fine-Tune Model

In [5]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import datetime

training_args = TrainingArguments(
    output_dir=f"./results/{model_name}-lora/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

# This step fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.683937,0.642000
2,No log,0.641104,0.767000
3,No log,0.342433,0.850000
4,0.604100,0.318412,0.860000
5,0.604100,0.308834,0.858000


TrainOutput(global_step=785, training_loss=0.4973863054992287, metrics={'train_runtime': 704.1429, 'train_samples_per_second': 35.504, 'train_steps_per_second': 1.115, 'total_flos': 3368485171200000.0, 'train_loss': 0.4973863054992287, 'epoch': 5.0})

### Step 6: Evalutaion the Foundation Model

In [6]:
original_performance = trainer.evaluate()

print("Original Model:", original_performance)

Original Model: {'eval_loss': 0.3184118866920471, 'eval_accuracy': 0.86, 'eval_runtime': 9.6912, 'eval_samples_per_second': 103.187, 'eval_steps_per_second': 3.302, 'epoch': 5.0}


###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [7]:
# Saving the model
model.save_pretrained(f"./models/{model_name}-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

### Step 1: Reload Base Model + Attach LoRA Adapter

In [8]:
from peft import AutoPeftModelForSequenceClassification
from peft import PeftModel

# Define label mappings
id2label = {0: 'negative', 1: 'positive'}
label2id = {'negative': 0, 'positive': 1}

loaded_model = AutoPeftModelForSequenceClassification.from_pretrained(
    f"./models/{model_name}-lora",
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Step 2: Evaluate on Validation Dataset

In [9]:
from transformers import Trainer, DataCollatorWithPadding

fine_tuned_trainer = Trainer(
    model=loaded_model,
    args=training_args,  # reuse your existing training_args (must run from top to bottom)
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

# Evaluate the fine-tuned (merged) model
fine_tuned_performance = fine_tuned_trainer.evaluate()

print("Fine-Tuned Model:", fine_tuned_performance)

Fine-Tuned Model: {'eval_loss': 0.3184118866920471, 'eval_model_preparation_time': 0.0009, 'eval_accuracy': 0.86, 'eval_runtime': 9.6553, 'eval_samples_per_second': 103.57, 'eval_steps_per_second': 3.314}
